In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os
import asyncio
import platform

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Function to extract pose features from a frame
def extract_pose_features(landmarks):
    if not landmarks or len(landmarks) < mp_pose.PoseLandmark.RIGHT_ANKLE.value + 1:
        return None

    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
    right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    left_ankle_angle = calculate_angle(left_knee, left_ankle, [left_ankle[0] + 0.1, left_ankle[1]])
    right_ankle_angle = calculate_angle(right_knee, right_ankle, [right_ankle[0] + 0.1, right_ankle[1]])

    features = {
        'Shoulder_Angle': left_shoulder_angle,
        'Elbow_Angle': left_elbow_angle,
        'Hip_Angle': left_hip_angle,
        'Knee_Angle': left_knee_angle,
        'Ankle_Angle': left_ankle_angle,
        'Shoulder_Ground_Angle': right_shoulder_angle,
        'Elbow_Ground_Angle': right_elbow_angle,
        'Hip_Ground_Angle': right_hip_angle,
        'Knee_Ground_Angle': right_knee_angle,
        'Ankle_Ground_Angle': right_ankle_angle
    }
    return features

# Process videos to create dataset
def create_dataset(dataset_path, output_csv):
    if not os.path.exists(dataset_path):
        print(f"Error: Dataset path {dataset_path} does not exist.")
        return pd.DataFrame()

    data = []
    labels = []
    processed_videos = 0
    total_frames = 0
    detected_poses = 0

    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        print(f"Checking: {class_dir}")
        if os.path.isdir(class_dir):
            for video_file in os.listdir(class_dir):
                if video_file.endswith(('.mp4', '.avi')):
                    video_path = os.path.join(class_dir, video_file)
                    print(f"Processing: {video_path}")
                    cap = cv2.VideoCapture(video_path)
                    if not cap.isOpened():
                        print(f"Error: Could not open {video_path}")
                        continue
                    frame_count = 0
                    pose_detected = False
                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret:
                            break
                        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        results = pose.process(image)
                        total_frames += 1
                        if results.pose_landmarks:
                            pose_detected = True
                            detected_poses += 1
                            features = extract_pose_features(results.pose_landmarks.landmark)
                            if features:
                                data.append(features)
                                labels.append(class_name.replace('_', ' '))
                        frame_count += 1
                        if frame_count > 50:
                            break
                    cap.release()
                    processed_videos += 1
                    print(f"{'Pose detected' if pose_detected else 'No pose detected'} in {video_file}")

    df = pd.DataFrame(data)
    df['Label'] = labels
    df.to_csv(output_csv, index=False)
    print(f"Processed {processed_videos} videos, {total_frames} frames, {detected_poses} poses detected")
    print("Column names:", df.columns.tolist() if not df.empty else "No data collected")
    return df

# Example usage
if __name__ == "__main__":
    dataset_path = "./dataset"  # Update with correct path relative to this script
    output_csv = "combined_pose_data.csv"
    df = create_dataset(dataset_path, output_csv)

Checking: ./dataset\.ipynb_checkpoints
Checking: ./dataset\combined_pose_data.csv
Checking: ./dataset\pull Up
Processing: ./dataset\pull Up\pull up_1.mp4
Pose detected in pull up_1.mp4
Processing: ./dataset\pull Up\pull up_10.mp4
Pose detected in pull up_10.mp4
Processing: ./dataset\pull Up\pull up_11.mp4
Pose detected in pull up_11.mp4
Processing: ./dataset\pull Up\pull up_12.mp4
Pose detected in pull up_12.mp4
Processing: ./dataset\pull Up\pull up_13.mp4
Pose detected in pull up_13.mp4
Processing: ./dataset\pull Up\pull up_14.mp4
Pose detected in pull up_14.mp4
Processing: ./dataset\pull Up\pull up_15.mp4
Pose detected in pull up_15.mp4
Processing: ./dataset\pull Up\pull up_16.mp4
Pose detected in pull up_16.mp4
Processing: ./dataset\pull Up\pull up_17.mp4
Pose detected in pull up_17.mp4
Processing: ./dataset\pull Up\pull up_18.mp4
Pose detected in pull up_18.mp4
Processing: ./dataset\pull Up\pull up_19.mp4
Pose detected in pull up_19.mp4
Processing: ./dataset\pull Up\pull up_2.mp4
P